In [1]:
from importlib import reload
import pandas as pd


In [2]:
import numpy as np
import spacy
from wordcloud import WordCloud


import matplotlib.pyplot as plt

In [3]:
import nltk
#nltk.download('stopwords')

In [4]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer


In [5]:
import sklearn.feature_extraction.text as text

In [6]:
import nlp_pandas_functions as npf
#dir(npf)

2022-02-23 14:32:38,118 loading file /home/cdsw/.flair/models/sentiment-en-mix-distillbert_4.pt


In [7]:
#pip3 install pyarrow
#pip3 install spacy
#pip3 install wordcloud
#pip3 install nltk
#pip3 install sklearn
#pip3 install ipdb

In [8]:
str_state = 'GA'
file = 'agent_comments_sales_'+str_state
#file = 'agent_comments_testdf'
#READ THE SALES DATA - STATE
df_sales = pd.read_feather(file + '_withembeddings.fea')
#df_sales = pd.read_feather(file+'_processed.fea')


In [9]:
df_sales_save = df_sales

In [10]:
df_sales = df_sales_save#.head(100000)

In [11]:
df_sales

,listingid,asgpropid,qtr,tax_year,cbsa_div,cbsa,state,Transdate,Transprice,bedrooms,...,clean_publicremarks_stemmed_additional_words_vaderpolarity,clean_publicremarks_lemmatized_vaderpolarity,clean_publicremarks_lemmatized_additional_words_vaderpolarity,sentence_composition,publicremarks_embeddings,clean_publicremarks_embeddings,clean_publicremarks_additional_words_embeddings,clean_publicremarks_nopunct_embeddings,clean_publicremarks_stemmed_embeddings,clean_publicremarks_lemmatized_embeddings
0,226661582,27649366.0,202008,2020.0,23580,23580,GA,20200811,359000.0,4.0,...,"{'compound': 0.4939, 'neg': 0.0, 'neu': 0.954,...","{'compound': 0.9578, 'neg': 0.0, 'neu': 0.786,...","{'compound': 0.93, 'neg': 0.0, 'neu': 0.81, 'p...","[55.00000000000001, 13.750000000000002, 17.5, ...","[-0.018459855, -0.023812085, -0.043892566, -0....","[-0.051987942, -0.054606345, -0.0580922, -0.03...","[-0.05043851, -0.056709077, -0.055227116, -0.0...","[-0.051987942, -0.054606345, -0.0580922, -0.03...","[-0.038664196, -0.05890688, -0.06525755, -0.02...","[-0.04986923, -0.05113033, -0.060150877, -0.03..."
1,226786904,27628370.0,202008,2020.0,29300,29300,GA,20200925,265000.0,4.0,...,"{'compound': 0.9686, 'neg': 0.0, 'neu': 0.799,...","{'compound': 0.9875, 'neg': 0.014, 'neu': 0.70...","{'compound': 0.9844, 'neg': 0.016, 'neu': 0.70...","[60.18518518518518, 13.88888888888889, 18.5185...","[-0.060909368, -0.044562507, -0.048587322, -0....","[-0.06049352, -0.05438737, -0.048537098, -0.05...","[-0.059828583, -0.05020684, -0.046291806, -0.0...","[-0.06049352, -0.05438737, -0.048537098, -0.05...","[-0.065291695, -0.06122866, -0.04930517, 0.006...","[-0.060174935, -0.05459462, -0.045611445, -0.0..."
2,226142320,27963359.0,202005,2020.0,27600,27600,GA,20200624,239900.0,4.0,...,"{'compound': 0.3182, 'neg': 0.0, 'neu': 0.957,...","{'compound': 0.8074, 'neg': 0.0, 'neu': 0.858,...","{'compound': 0.8074, 'neg': 0.0, 'neu': 0.855,...","[60.37735849056604, 9.433962264150944, 24.5283...","[0.051360507, -0.007422204, -0.054501485, -0.0...","[-0.015618877, -0.03321094, -0.064131856, -0.0...","[-0.011177328, -0.034213364, -0.06423391, -0.0...","[-0.015618877, -0.03321094, -0.064131856, -0.0...","[-0.026406875, -0.061117634, -0.071621135, -0....","[-0.026331514, -0.048204094, -0.06567335, -0.0..."
3,226738459,27516267.0,202008,2020.0,29300,29300,GA,20200713,130900.0,3.0,...,"{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos...","{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos...","{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos...","[53.84615384615385, 7.6923076923076925, 38.461...","[-0.020899976, 0.016856506, -0.011256389, -0.0...","[-0.028272344, -0.005281781, -0.0073222406, -0...","[0.0036934454, 0.047616564, -0.0056924997, 0.0...","[-0.028272344, -0.005281781, -0.0073222406, -0...","[-0.050069433, 0.007041185, -0.033259477, -0.0...","[-0.028272344, -0.005281781, -0.0073222406, -0..."
4,226850183,27791872.0,202008,2020.0,23580,23580,GA,20200702,406000.0,4.0,...,"{'compound': 0.2023, 'neg': 0.0, 'neu': 0.969,...","{'compound': 0.802, 'neg': 0.0, 'neu': 0.876, ...","{'compound': 0.6908, 'neg': 0.0, 'neu': 0.908,...","[65.57377049180327, 13.114754098360656, 11.475...","[-0.047313046, 0.00859506, -0.04837404, -0.052...","[-0.055047132, -0.031499583, -0.055006824, -0....","[-0.053909175, -0.025725972, -0.05363282, -0.0...","[-0.055047132, -0.031499583, -0.055006824, -0....","[-0.061100535, -0.05123385, -0.067388244, -0.0...","[-0.054386344, -0.04275706, -0.052802, -0.0314..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633866,77898274,106321613.0,200708,2013.0,27600,27600,GA,20070719,163900.0,NaN,...,"{'compound': 0.5574, 'neg': 0.0, 'neu': 0.839,...","{'compound': 0.5574, 'neg': 0.0, 'neu': 0.85, ...","{'compound': 0.5574, 'neg': 0.0, 'neu': 0.839,...","[57.14285714285714, 17.857142857142858, 14.285...","[-0.051895734, -0.034251053, 0.033765092, -0.0...","[-0.060446255, -0.039059404, -0.010817213, 0.0...","[-0.05962046, -0.036440928,

In [12]:
df_sales['transaction_year'] = np.floor(np.array((df_sales['Transdate'].values/10000)))

In [13]:
df_sales.columns

Index(['listingid', 'asgpropid', 'qtr', 'tax_year', 'cbsa_div', 'cbsa',
       'state', 'Transdate', 'Transprice', 'bedrooms', 'total_value',
       'cj_living_area', 'basement', 'has_pool', 'parking', 'is_poor',
       'is_good', 'HAS_VIEW', 'has_golf', 'has_water', 'has_woods', 'has_hill',
       'fips_code', 'census_tract', 'zip', 'effective_year_built', 'story',
       'ListingPrice', 'publicremarks', 'avmValue', 'avmerror', 'rentsale',
       'publicremarks_prepared', 'publicremarks_prepared_unicode',
       'clean_publicremarks', 'clean_publicremarks_additional_words',
       'clean_publicremarks_nopunct',
       'clean_publicremarks_nopunct_additional_words',
       'clean_publicremarks_stemmed',
       'clean_publicremarks_stemmed_additional_words',
       'clean_publicremarks_lemmatized',
       'clean_publicremarks_lemmatized_additional_words',
       'publicremarks_nltkpolarity', 'clean_publicremarks_nltkpolarity',
       'clean_publicremarks_additional_words_nltkpolarity',


In [14]:
print(df_sales['transaction_year'].min())
print(df_sales['transaction_year'].max())

2006.0
2021.0


In [15]:
from nlp_pandas_functions import ngram_analysis
reload(ngram_analysis)
dir(ngram_analysis)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'bigrams_join',
 'count_n_grams',
 'defaultdict',
 'df_get_n_grams_count',
 'df_lambda_ngram',
 'generate_n_grams',
 'get_imp',
 'get_n_grams_count',
 'get_n_grams_probability',
 'kg',
 'ngrams',
 'np',
 'pd',
 'text']

In [16]:
dir(ngram_analysis)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'bigrams_join',
 'count_n_grams',
 'defaultdict',
 'df_get_n_grams_count',
 'df_lambda_ngram',
 'generate_n_grams',
 'get_imp',
 'get_n_grams_count',
 'get_n_grams_probability',
 'kg',
 'ngrams',
 'np',
 'pd',
 'text']

In [21]:
def get_n_grams_probability(strtext, ngram, ngram_df):
    df_ngram = ngram_analysis.get_n_grams_count(strtext,ngram)
    df_ngram = df_ngram.rename({0: 'ngram', 1: 'count'}, axis='columns')
    df_process = pd.merge(df_ngram, ngram_df, on=["ngram"])
    df_process['probability_mult'] = df_process['probability'] * df_process['count_x']
    mean_prob =  (df_process['probability_mult'].sum() / df_process['count_x'].sum())
    ngram_probs = list(df_process['probability'].quantile([.99, 0.95, 0.9, 0.75, 0.5, 0.25]))
    ngram_probs.append(mean_prob)
    return ngram_probs, df_process

In [22]:
import time

In [ ]:
columns = ['publicremarks', 'clean_publicremarks', 'clean_publicremarks_additional_words',
          'clean_publicremarks_nopunct', 'clean_publicremarks_nopunct_additional_words',
           'clean_publicremarks_stemmed', 'clean_publicremarks_stemmed_additional_words', 
           'clean_publicremarks_lemmatized', 'clean_publicremarks_lemmatized_additional_words'
           ]

In [ ]:
columns = ['clean_publicremarks', 'clean_publicremarks_additional_words',
          'clean_publicremarks_nopunct', 'clean_publicremarks_nopunct_additional_words',
           'clean_publicremarks_stemmed', 'clean_publicremarks_stemmed_additional_words', 
           'clean_publicremarks_lemmatized', 'clean_publicremarks_lemmatized_additional_words'
           ]

suffix_1 = ['_ngram_probs_1_99', '_ngram_probs_1_95', '_ngram_probs_1_90', 
            '_ngram_probs_1_75', '_ngram_probs_1_50', '_ngram_probs_1_25', '_ngram_probs_1_MN']
suffix_2 = ['_ngram_probs_2_99', '_ngram_probs_2_95', '_ngram_probs_2_90', 
            '_ngram_probs_2_75', '_ngram_probs_2_50', '_ngram_probs_2_25', '_ngram_probs_2_MN']
suffix_3 = ['_ngram_probs_3_99', '_ngram_probs_3_95', '_ngram_probs_3_90', 
            '_ngram_probs_3_75', '_ngram_probs_3_50', '_ngram_probs_3_25', '_ngram_probs_3_MN']
suffix_4 = ['_ngram_probs_4_99', '_ngram_probs_4_95', '_ngram_probs_4_90', 
            '_ngram_probs_4_75', '_ngram_probs_4_50', '_ngram_probs_4_25', '_ngram_probs_4_MN']


for column in columns:
    print(column)
    
    tic = time.clock()
    unigram_df = pd.read_feather('ngrams/' + file + '_withsentiment_' + column + '_unigram.fea')
    bigram_df = pd.read_feather('ngrams/' + file + '_withsentiment_' + column + '_bigram.fea')
    trigram_df = pd.read_feather('ngrams/' + file + '_withsentiment_' + column + '_trigram.fea')
    fourgram_df = pd.read_feather('ngrams/' + file + '_withsentiment_' + column + '_fourgram.fea')
    
    toc_r = time.clock()
    print('Reading..',toc_r - tic)
    
    ngram_probs_array_uni = np.zeros([len(df_sales), 7])  
    ngram_probs_array_bi = np.zeros([len(df_sales), 7])
    ngram_probs_array_tri = np.zeros([len(df_sales), 7])
    ngram_probs_array_four = np.zeros([len(df_sales), 7])
    
    column_names_1 = []
    for cn in suffix_1:
        column_names_1.append(column + cn)
        
    column_names_2 = []
    for cn in suffix_2:
        column_names_2.append(column + cn)
        
    column_names_3 = []
    for cn in suffix_3:
        column_names_3.append(column + cn)
        
    column_names_4 = []
    for cn in suffix_4:
        column_names_4.append(column + cn)
        
    for house in df_sales.index:
        print(house, len(df_sales.index))
                   
        tic = time.clock()
        ngram_probs, df_process = get_n_grams_probability(df_sales[column][house], 
                                                          1, unigram_df)
        ngram_probs_array_uni[house, :] = ngram_probs

        tic = time.clock()
        ngram_probs, df_process = get_n_grams_probability(df_sales[column][house], 
                                                          2, bigram_df)
        ngram_probs_array_bi[house, :] = ngram_probs

        tic = time.clock()
        ngram_probs, df_process = get_n_grams_probability(df_sales[column][house], 
                                                          3, trigram_df)
        ngram_probs_array_tri[house, :] = ngram_probs

        tic = time.clock()
        ngram_probs, df_process = get_n_grams_probability(df_sales[column][house], 
                                                          4, fourgram_df)
        ngram_probs_array_four[house, :] = ngram_probs 

    for cid, colum_id in enumerate(column_names_3):
        df_sales[colum_id] = ngram_probs_array_tri[:, cid].tolist() 
        
    for cid, colum_id in enumerate(column_names_1):
        df_sales[colum_id] = ngram_probs_array_uni[:, cid].tolist()   

    for cid, colum_id in enumerate(column_names_2):
        df_sales[colum_id] = ngram_probs_array_bi[:, cid].tolist() 

    for cid, colum_id in enumerate(column_names_4):
        df_sales[colum_id] = ngram_probs_array_four[:, cid].tolist() 
        
    toc = time.clock()
    print(toc - tic)

    

publicremarks
Reading.. 29.528386999999995
0 636149
1 636149
2 636149
3 636149


In [ ]:
df_sales.to_feather(file + '_withsentiment_sentencecomposition_' + 'ngram_probability' + '.fea')

In [ ]:
df_sales[column][0]

In [ ]:
df_sales


In [ ]:
#column = 'clean_publicremarks_nopunct'

In [ ]:
#unigram_df = pd.read_feather('ngrams/' + file + '_withsentiment_' + column + '_unigram.fea')
#bigram_df = pd.read_feather('ngrams/' + file + '_withsentiment_' + column + '_bigram.fea')
#trigram_df = pd.read_feather('ngrams/' + file + '_withsentiment_' + column + '_trigram.fea')
#fourgram_df = pd.read_feather('ngrams/' + file + '_withsentiment_' + column + '_fourgram.fea')

In [ ]:
#unigram_df[unigram_df['ngram'] == 'house']['probability']

In [ ]:
house = 5000

In [ ]:
print(df_sales['publicremarks'].values[house])
print(df_sales['ListingPrice'].values[house])
print(df_sales['avmValue'].values[house])
print(df_sales['Transprice'].values[house])
print(df_sales['avmerror'].values[house])
print((df_sales['Transprice'].values[house]-(df_sales['avmValue'].values[house]))/df_sales['Transprice'].values[house])
print((df_sales['Transprice'].values[house]-(df_sales['ListingPrice'].values[house]))/df_sales['Transprice'].values[house])

In [ ]:
df_sales['clean_publicremarks_nopunct'].values[house]

In [ ]:
df_process = get_n_grams_probability(df_sales['clean_publicremarks_nopunct'].values[house], 2, bigram_df)

In [ ]:
ngram_analysis.get_n_grams_count(df_sales['clean_publicremarks_nopunct'].values[house],2)

In [ ]:
print(mean_prob)

In [ ]:
df_process

In [ ]:
df_process.probability.mean()

In [ ]:
df_process.probability.std()

In [ ]:
unigram_df = unigram_df.rename({0: 'ngram', 1: 'count'}, axis='columns')